In [28]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy import signal
from scipy.optimize import minimize

import random

import h5py
from IPython.utils import io

from joblib import Parallel, delayed
from tqdm import tqdm
import sys

In [29]:
%matplotlib widget

%reload_ext autoreload
%autoreload 2

In [30]:
sys.path.append('../../../../nadavp/new_trap/Tools/')
sys.path.append('../../../../nadavp/new_trap/Tools/StatFramework/')
sys.path.append('../../../../nadavp/opt_lev_analysis/lib/')
import BeadDataFile
from discharge_tools import *

In [31]:
from likelihood_calculator import likelihood_analyser
from likelihood_calculator import GravityFramework
from likelihood_calculator import auxiliary_functions as aux
gfw = GravityFramework.GravityFramework()

In [32]:
import bead_util as bu
# import calib_util as cal
import transfer_func_util as tf
import configuration as config

### Discharge data

In [33]:
step_cal_dir = r'/data/old_trap/20211001/bead1/discharge/recharge_20211013/'
max_file = 1

In [34]:
step_cal_files = []
for root, dirnames, filenames in os.walk(step_cal_dir):
    for filename in fnmatch.filter(filenames, '*' + config.extensions['data']):
        if '_fpga.h5' in filename:
            continue
        step_cal_files.append(os.path.join(root, filename))
step_cal_files.sort(key = bu.find_str)
num_calib_files = len(step_cal_files)
print(num_calib_files)

# Do the step calibration

step_file_objs = []
for filname in step_cal_files[:]:
    try:
        df = bu.DataFile();
        df.load(filname);
        df.load_other_data()
        step_file_objs.append(df);
    except:
        print('***')

175


In [35]:
scaleY = 97.9992158717
scaleZ = scaleY*(418+627)/(392+406)
scaleX = scaleY*(421+368)/(392+406)
scaleX, scaleY, scaleZ

(96.89396155735751, 97.9992158717, 128.33230649865476)

In [36]:
print("Scales are ", scaleY)
bb = step_file_objs[-1]
drive_freq = 139
bandwidth=2
fsamp = 5000
b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
xx2 = signal.filtfilt(b, a, bb.other_data[5])
voltage = np.std(xx2)*np.sqrt(2)*100
print("voltage: ", voltage)

Scales are  97.9992158717
voltage:  24.0265126779


In [37]:
i = 0
fname = r'/data/old_trap/20211001/bead1/neutrality9/Z_200Vpp_alternate_139Hz_spin_XY_117kHz_175Vpp_delay_4sec_with_height_fb_redo5/Z_200Vpp_'+str(i)+'.h5'
bb = bu.DataFile();
bb.load(fname);
bb.load_other_data()
print("voltage: ", voltage, np.mean(bb.other_data[1])*100, np.mean(bb.other_data[2])*100)
drive_freq = 139
bandwidth=2
b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
xx2 = signal.filtfilt(b, a, bb.other_data[1])
voltage_200_1 = np.std(xx2)*np.sqrt(2)*100
xx3 = signal.filtfilt(b, a, bb.other_data[2])
voltage_200_2 = np.std(xx3)*np.sqrt(2)*100
print("voltage: ", voltage_200_1, voltage_200_2)

voltage:  24.0265126779 -0.592538948653 -0.574958113985
voltage:  190.87201524 0.674330527298


In [38]:
etha = 1/scaleZ*327*1.6e-19*voltage
to_epsilon = 1/(2*327*1.6e-19*voltage_200_1*(418)/(418+627))

### Look at neutrality data

In [39]:
def get_amp_wraperZ(i, fname):
    try:
        return get_ampZ(i, fname)
    except:
        return 0,0,0,0

In [274]:
ll = likelihood_analyser.LikelihoodAnalyser()
bandwidth = 3
decimate = 2
def get_ampZ(i, foldername):
    
    fname = foldername+'Z_200Vpp_'+str(i)+'.h5'
    drive_freq = 139
    bb = bu.DataFile();
    bb.load(fname);
    bb.load_other_data()
    
    jj = i%2
    freq2 = np.fft.rfftfreq(len(bb.other_data[1+jj]), d=1./5000)
    fft_angles = np.angle(np.fft.rfft(bb.other_data[1+jj]))
    phi_tmp2 = (fft_angles[freq2==drive_freq]+np.pi/2)%(2*np.pi)

    phase_diff = 4.8        
    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': phi_tmp2+phase_diff, 
                  'error_A': 1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1,
                  'limit_A': [-100000, 100000], 'fix_f': True, 'fix_phi': True,
                  'print_level': 0}
    
    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': phi_tmp2+phase_diff, 'sigma': 1100,
                  'error_A': 0.1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1, 'error_sigma':50,
                  'limit_A': [-100000, 100000], 'limit_sigma':[100,3000000], 'fix_phi': True, 'fix_f': True,
                  'print_level': 0}

    b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
    xx2 = signal.filtfilt(b, a, bb.pos_data[2])[::decimate]
    m1_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
    
    phase_diff = 6   
    drive_freq = drive_freq*2+1
    
    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': phi_tmp2+phase_diff, 
                  'error_A': 1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1,
                  'limit_A': [-1000000, 1000000], 'fix_f': True, 'fix_phi': False,
                  'print_level': 0}

    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': phi_tmp2+phase_diff, 'sigma': 1100,
                  'error_A': 0.1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1, 'error_sigma':50,
                  'limit_A': [-100000, 100000], 'limit_sigma':[100,3000000], 'fix_phi': False, 'fix_f': True,
                  'print_level': 0}
    
    b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
    xx2 = signal.filtfilt(b, a, bb.pos_data[2])[::decimate]
    m2_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)

    return m1_tmp.values[0], m2_tmp.values[0], m1_tmp.values[3], (m2_tmp.values[2]-phi_tmp2)%(2*np.pi)

In [263]:
fname = base_folder+r'neutrality6/Z_150Vpp_alternate_139Hz_spin_XY_93kHz_200Vpp_delay_3sec_with_height_fb/'
get_ampZ(50, fname)

(-14.783051875900128,
 -43.10113857971737,
 4389.2410302933185,
 array([ 0.66915054]))

In [162]:
amps_tmp = np.array(Parallel(n_jobs=40)(delayed(get_amp_wraperZ)(i, fname) for i in tqdm(range((2200)))))

100%|██████████| 2200/2200 [00:28<00:00, 191.23it/s]
/home/analysis_user/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


In [275]:
base_folder = r'/data/old_trap/20211001/bead1/'
fnames = [base_folder+r'neutrality6/Z_150Vpp_alternate_139Hz_spin_XY_93kHz_200Vpp_delay_3sec_with_height_fb/',
base_folder+r'neutrality6/Z_150Vpp_alternate_139Hz_spin_XY_103kHz_200Vpp_delay_3sec_with_height_fb/',
base_folder+r'neutrality6/Z_150Vpp_alternate_139Hz_spin_XY_103kHz_200Vpp_delay_3sec_with_height_fb_redo/',
# base_folder+r'neutrality5/Z_150Vpp_alternate_139Hz_spin_XY_103kHz_200Vpp_delay_3sec_with_height_fb_redo2/', ## crashed
base_folder+r'neutrality6/Z_200Vpp_alternate_139Hz_spin_XY_103kHz_175Vpp_delay_3sec_with_height_fb_redo3/',
base_folder+r'neutrality6/Z_200Vpp_alternate_139Hz_spin_XY_103kHz_175Vpp_delay_3sec_with_height_fb_redo4/',
# base_folder+r'neutrality7/Z_200Vpp_alternate_139Hz_spin_XY_103kHz_175Vpp_delay_3sec_with_height_fb/' ## crashed
         ]

files = [2000, 2000, 2000, 1000, 1000]

In [276]:
ams_neutrality67 = []
for i_folder in range(len(files)):
    amps_tmp = np.array(Parallel(n_jobs=40)(delayed(get_amp_wraperZ)(i, fnames[i_folder]) for i in tqdm(range((files[i_folder])))))
    ams_neutrality67 += list(np.array(amps_tmp)*etha)
ams_neutrality67 = np.array(ams_neutrality67)


100%|██████████| 2000/2000 [00:53<00:00, 37.37it/s]/home/analysis_user/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  This is separate from the ipykernel package so we can avoid doing imports until

 86%|████████▌ | 1720/2000 [00:45<00:07, 36.88it/s]


100%|██████████| 1000/1000 [00:26<00:00, 36.39it/s]

In [277]:
ams_neutrality67_2 = list(ams_neutrality67)
ams_neutrality67_2.pop(692)
ams_neutrality67_2.pop(692)
ams_neutrality67 = np.array(ams_neutrality67_2)

In [278]:
base_folder = r'/data/old_trap/20211001/bead1/'
fnames = [base_folder+r'neutrality8/Z_200Vpp_alternate_139Hz_spin_XY_117kHz_175Vpp_delay_5sec_with_height_fb/',
# base_folder+r'neutrality8/Z_200Vpp_alternate_139Hz_spin_XY_117kHz_175Vpp_delay_5sec_with_height_fb_redo/', ## 4 sec.
base_folder+r'neutrality8/Z_200Vpp_alternate_139Hz_spin_XY_117kHz_175Vpp_delay_7sec_with_height_fb_redo2/',
base_folder+r'neutrality8/Z_200Vpp_alternate_139Hz_spin_XY_117kHz_175Vpp_delay_7sec_with_height_fb_redo3/',
base_folder+r'neutrality8/Z_200Vpp_alternate_139Hz_spin_XY_117kHz_175Vpp_delay_7sec_with_height_fb_redo4/',
base_folder+r'neutrality8/Z_200Vpp_alternate_139Hz_spin_XY_117kHz_175Vpp_delay_7sec_with_height_fb_redo5/']

files = [2000, 2000, 2000, 2000, 440]

In [279]:
ams_neutrality8 = []
for i_folder in range(len(files)):
    amps_tmp = np.array(Parallel(n_jobs=40)(delayed(get_amp_wraperZ)(i, fnames[i_folder]) for i in tqdm(range((files[i_folder])))))
    ams_neutrality8 += list(np.array(amps_tmp)*etha)
ams_neutrality8 = np.array(ams_neutrality8)


100%|██████████| 2000/2000 [00:55<00:00, 37.03it/s]/home/analysis_user/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  This is separate from the ipykernel package so we can avoid doing imports until

 86%|████████▌ | 1720/2000 [00:41<00:06, 41.47it/s]


100%|██████████| 440/440 [00:08<00:00, 46.80it/s]

In [280]:
base_folder = r'/data/old_trap/20211001/bead1/'
fnames = [base_folder+r'neutrality9/Z_200Vpp_alternate_139Hz_spin_XY_117kHz_175Vpp_delay_4sec_with_height_fb/',
base_folder+r'neutrality9/Z_200Vpp_alternate_139Hz_spin_XY_117kHz_175Vpp_delay_4sec_with_height_fb_redo/',
base_folder+r'neutrality9/Z_200Vpp_alternate_139Hz_spin_XY_117kHz_175Vpp_delay_4sec_with_height_fb_redo2/',
base_folder+r'neutrality9/Z_200Vpp_alternate_139Hz_spin_XY_117kHz_175Vpp_delay_4sec_with_height_fb_redo3/',
base_folder+r'neutrality9/Z_200Vpp_alternate_139Hz_spin_XY_117kHz_175Vpp_delay_6sec_with_height_fb_redo4/',
base_folder+r'neutrality9/Z_200Vpp_alternate_139Hz_spin_XY_117kHz_175Vpp_delay_4sec_with_height_fb_redo5/',
base_folder+r'neutrality9/Z_200Vpp_alternate_139Hz_spin_XY_117kHz_175Vpp_delay_6sec_with_height_fb_redo6/']

files = [80, 2000, 2000, 2000, 2000, 4000, 2200]

In [281]:
ams_neutrality9 = []
for i_folder in range(len(files)):
    amps_tmp = np.array(Parallel(n_jobs=40)(delayed(get_amp_wraperZ)(i, fnames[i_folder]) for i in tqdm(range((files[i_folder])))))
    ams_neutrality9 += list(np.array(amps_tmp)*etha)
ams_neutrality9 = np.array(ams_neutrality9)


100%|██████████| 80/80 [00:00<00:00, 100.91it/s]/home/analysis_user/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  This is separate from the ipykernel package so we can avoid doing imports until

 82%|████████▏ | 1640/2000 [00:45<00:10, 35.66it/s]


 91%|█████████ | 3640/4000 [01:38<00:09, 36.26it/s]


100%|██████████| 2200/2200 [00:59<00:00, 36.27it/s]

In [209]:
## different spin voltage -- skip that
# base_folder = r'/data/old_trap/20211001/bead1/'
# fnames = [base_folder+r'neutrality11/Z_200Vpp_alternate_139Hz_spin_XY_117kHz_200Vpp_delay_6sec_with_height_fb/']
# 
# files = [2500]

In [210]:
# ams_neutrality11 = []
# for i_folder in range(len(files)):
#     amps_tmp = np.array(Parallel(n_jobs=40)(delayed(get_amp_wraperZ)(i, fnames[i_folder]) for i in tqdm(range((files[i_folder])))))
#     ams_neutrality11 += list(np.array(amps_tmp)*etha)
# ams_neutrality11 = np.array(ams_neutrality11)

In [282]:
amps_all2 = np.array(list(ams_neutrality67)+list(ams_neutrality8)+list(ams_neutrality9))

In [134]:
data = amps_all[::2,0]
data2 = amps_all[1::2,0]
data3 = data-0.645*data2

data_1 = np.abs(amps_all[::2,1])
data2_1 = np.abs(amps_all[1::2,1])
data3_1 = data_1+0.645*0.645*data2_1

_, ax= plt.subplots(figsize=(9.5,4))
ax.scatter(np.arange(len(data))*2*10/3600, data, label='200Vpp-z-plus, xy-spin-50Vpp-103kHz')
ax.scatter(np.arange(len(data))*2*10/3600, data2, label='200Vpp-z-minus, xy-spin-50Vpp-103kHz', alpha=0.6)
# ax.scatter(np.arange(len(data3))*2*10/3600, data3, label='200Vpp-z, A parameter', alpha=0.6)

ax.plot(np.arange(len(data3))*2*10/3600, np.arange(len(data3))*0, '--k')
ax.set(xlabel='Integration time [hr.]', ylabel=r'$\epsilon$')
ax.legend()
print('1st harmonic, electrode 1: mean, std: ', np.mean(data), np.std(data)/np.sqrt(len(data)))
print('1st harmonic, electrode 2: mean, std: ', np.mean(data2), np.std(data2)/np.sqrt(len(data2)))
print('1st harmonic, A parameter: mean, std: ', np.mean(data3), np.std(data3)/np.sqrt(len(data3)))
print('1st harmonic, A parameter[epsilon]: mean, std: ', np.mean(data3)*to_epsilon, to_epsilon*np.std(data3)/np.sqrt(len(data3)))

print()
print('2nd harmonic, electrode 1: mean, std: ', np.mean(data_1), np.std(data_1)/np.sqrt(len(data_1)))
print('2nd harmonic, electrode 2: mean, std: ', np.mean(data2_1), np.std(data2_1)/np.sqrt(len(data2_1)))
print('2nd harmonic, B parameter: mean, std: ', np.mean(data3_1), np.std(data3_1)/np.sqrt(len(data3_1)))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

1st harmonic, electrode 1: mean, std:  -2.1770774109482787e-17 6.36621564849e-19
1st harmonic, electrode 2: mean, std:  -3.4006400142683797e-17 8.26668817635e-19
1st harmonic, A parameter: mean, std:  1.6335398254823102e-19 4.08094746576e-19
1st harmonic, A parameter[epsilon]: mean, std:  2.04470070078e-05 5.10811919791e-05

2nd harmonic, electrode 1: mean, std:  2.6225722784247494e-16 1.03524641174e-18
2nd harmonic, electrode 2: mean, std:  5.314550594329019e-16 2.07933439476e-18
2nd harmonic, B parameter: mean, std:  4.833558189430473e-16 1.35209130461e-18


In [283]:
fig, ax = plt.subplots()

# data_11 = amps_all[::2,1]
# data_12 = amps_all[::2,1] ## 4.8-pi/2
# data_13 = amps_all[::2,1] ## 4.8
# data_14 = amps_all[::2,1] ## 4.8+pi/2
data_15 = amps_all[::2,1]## no phase 
data_16 = amps_all[::2,1]## sideband

# _ = ax.hist(2*data_11/(1.1e5*200)/1.6e-19*1e6, range=(-300,300), bins=100)
# _ = ax.hist(2*data_12/(1.1e5*200)/1.6e-19*1e6, range=(-300,300), bins=100)
# _ = ax.hist(2*data_13/(1.1e5*200)/1.6e-19*1e6, range=(-300,300), bins=100)
# _ = ax.hist(2*data_14/(1.1e5*200)/1.6e-19*1e6, range=(-300,300), bins=100)
_ = ax.hist(2*data_15/(1.1e5*200)/1.6e-19*1e6, range=(-300,300), bins=100)
_ = ax.hist(2*data_16/(1.1e5*200)/1.6e-19*1e6, range=(-300,300), bins=100)
# _ = ax.hist(2*data_phase2/(1.1e5*200)/1.6e-19*1e6, range=(-300,300), bins=100)
ax.set(xlabel='$p_{ac}$ [e $\mu$m]')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[Text(0.5, 0, '$p_{ac}$ [e $\\mu$m]')]

In [273]:
data_phase = (amps_all[0::2,3]/etha)%(np.pi) ## 4.8+pi/2
data_phase2 = 2*amps_all[::2,1]/(1.1e5*200)/1.6e-19*1e6 ## 4.8+pi/2
data_phase3 = ams_neutrality67[::2,1] ## 4.8+pi/2

In [258]:
data_phase2

array([236.44047117415414, -232.96567588985457, -229.7111158246102, ...,
       -231.66826581855193, -211.65498366652804, -238.6539125080112], dtype=object)

In [259]:
_, ax = plt.subplots()
# _ = ax.scatter(range(len(data_phase)), data_phase)
_ = ax.scatter(data_phase2, data_phase)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [174]:
np.save("all_200Vpp_ol_trap_alternate_sideband", amps_all)

In [27]:
N = len(data)
data_shifted = np.array([(data[i]+data[i-1])/2 for i in range(1,N)])
data3_shifted = data_shifted-0.645*data2[:-1]

_, ax= plt.subplots(figsize=(9.5,4))
ax.scatter(np.arange(len(data3))*2*10/3600, data3, label='200Vpp-z-plus, A parameter')
ax.scatter(np.arange(len(data3_shifted))*2*10/3600, data3_shifted, label='200Vpp-z, corrected A parameter', alpha=0.6)

ax.plot(np.arange(len(data3))*2*10/3600, np.arange(len(data3))*0, '--k')
ax.set(xlabel='Integration time [hr.]', ylabel=r'$\epsilon$')
ax.legend()
print('1st harmonic, A parameter: mean, std: ', np.mean(data3), np.std(data3)/np.sqrt(len(data3)))
print('1st harmonic, corrected A: mean, std: ', np.mean(data3_shifted), np.std(data3_shifted)/np.sqrt(len(data3_shifted)))
print('1st harmonic, A parameter[epsilon]: mean, std: ', np.mean(data3_shifted)*to_epsilon, to_epsilon*np.std(data3_shifted)/np.sqrt(len(data3)))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

1st harmonic, A parameter: mean, std:  6.44059727924e-19 4.0429949334e-19
1st harmonic, corrected A: mean, std:  6.44350521948e-19 3.24537364129e-19
1st harmonic, A parameter[epsilon]: mean, std:  8.0653311491e-05 4.06209975897e-05


### Fitting

In [ ]:
from scipy.optimize import curve_fit
from scipy import asarray as ar,exp

n_bins = 29
range_pram = 0.02
_,ax = plt.subplots()
entries, bins, patches = ax.hist(data3*to_epsilon, bins=n_bins, range=(-range_pram, range_pram), label='200Vpp, all frequencies')
# entries3, bins, patches = ax.hist(data2_1, bins=n_bins, label='150Vpp, 77kHz', alpha=0.6)
ax.set(xlabel=r'Force [N]')
# ax.legend()

In [ ]:
data_new_trap = np.load('tmp.npy')

In [ ]:
np.mean(data_hist), np.std(data_hist)/np.sqrt(len(data_hist))

In [ ]:
n_bins = 21
range_pram = 0.015
data_hist = np.array(list(data_new_trap)+list(data3_shifted*to_epsilon))
_,ax = plt.subplots()
entries, bins, patches = ax.hist(data_hist, bins=n_bins, range=(-range_pram, range_pram), label='200Vpp, all frequencies')
# entries3, bins, patches = ax.hist(data3, bins=n_bins, range=(-range_pram, range_pram), label='150Vpp, 77kHz', alpha=0.6)
ax.set(xlabel=r'$\epsilon$')
ax.legend()

In [ ]:
## gaussian fit to the histogram

bins_fit =[(bins[i+1]-bins[i])/2+bins[i] for i in range(len(entries))]

titles = ['', '', '']

for i,ee in enumerate([entries]):
    x = bins_fit
    y= np.array(ee)
    sigma_bin = np.sqrt(ee)
    
    n = len(x)                          #the number of data
    mean = 0                  #note this correction
    sigma = 0.03       #note this correction

    def gaus(x,a,x0,sigma):
        return a*np.exp(-(x-x0)**2/(2*sigma**2))

    popt,pcov = curve_fit(gaus,x,y,p0=[1,mean,sigma])
    ax.plot(x,gaus(x,*popt),'ro:',label='fit')

    print("mean: ", popt[1], " ,std (on the mean): ", popt[2]/np.sqrt(len(data)), " ,sigma: ", popt[2])
    print("std from the covariance matrix: ", np.sqrt(pcov[1,1]))
    print('\nNeutrality: ', 1/1.26e14*popt[2]/np.sqrt(len(data)))
    print('Factor needed for neutrality best limit: ', 1/1.26e14*popt[2]/np.sqrt(len(data))/1e-21)

In [ ]:
z_top_means = []
z_bottom_mean = []
z_top_std = []
z_bottom_std = []

bin_size = 10
bins = len(data)//bin_size
for i in range(bins):
    z_top_means.append(np.mean(data[i*bin_size:(i+1)*bin_size]))
    z_bottom_mean.append(np.mean(data2[i*bin_size:(i+1)*bin_size]))
    z_top_std.append(np.std(data[i*bin_size:(i+1)*bin_size]))
    z_bottom_std.append(np.std(data2[i*bin_size:(i+1)*bin_size]))
z_top_means = np.array(z_top_means)
z_top_std = np.array(z_top_std)
z_bottom_mean = np.array(z_bottom_mean)
z_bottom_std = np.array(z_bottom_std)

In [ ]:
thresh = 0.015
data3_eps = data3_shifted*to_epsilon
data3_eps = data_hist
thresh=3
data3_filt = [dd for dd in data3_eps if dd<thresh if dd>-thresh]
data3_filt = data_hist
print(len(data3), len(data3_filt))
print((len(data3_filt)-len(data3))/len(data3)*-100)

In [ ]:
z_combine_means = []
z_combine_std = []

bin_size = 20
bins = len(data3_filt)//bin_size
for i in range(bins):
    z_combine_means.append(np.mean(data3_filt[i*bin_size:(i+1)*bin_size]))
    z_combine_std.append(np.std(data3_filt[i*bin_size:(i+1)*bin_size]))
z_combine_means = np.array(z_combine_means)
z_combine_std = np.array(z_combine_std)/np.sqrt(bin_size)

In [175]:
_,ax = plt.subplots(figsize=(9.4,4))
iteration = np.arange(len(z_combine_means))*10*bin_size*2/3600
ax.plot(iteration, iteration*0, 'r--')
top_err = z_top_std/np.sqrt(bin_size)
bottom_err = z_bottom_std/np.sqrt(bin_size)
A = z_top_means-z_bottom_mean*0.66
A_err = np.sqrt(top_err**2+top_err**2*0.66*0.66)
# ax.errorbar(iteration, z_top_means, yerr=top_err, fmt='o', label='top electrode')
# ax.errorbar(iteration, z_bottom_mean*0.66, yerr=bottom_err, fmt='o', label='bottom electrode')
# ax.errorbar(iteration, A, yerr=A_err, fmt='o', label='A parameter')
ax.errorbar(iteration, z_combine_means, yerr=z_combine_std, fmt='o', label='A parameter', alpha=0.3)
# ax.errorbar(iteration, z_combine_means_2, yerr=z_combine_std_2, fmt='o', label='A parameter', alpha=0.3)
ax.legend()
ax.set(xlabel='Measuring time [hr.]', ylabel='Measured force [N]')
# ax.errorbar(iteration, (z_top_means-z_bottom_mean*0.66)*to_epsilon, yerr=z_bottom_std/np.sqrt(10)*to_epsilon, fmt='o')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

NameError: name 'z_combine_means' is not defined

In [ ]:
A_epsion = z_combine_means
A_err_epsilon = z_combine_std
def calculate_pl(force_):
    chi2 = np.power((A_epsion-force_)/A_err_epsilon, 2)
    return np.sum(chi2)

In [ ]:
_,ax = plt.subplots()
pl_range = np.arange(-3e-4,3e-4,2e-6)
pl = np.array([calculate_pl(pl_range_) for pl_range_ in pl_range])
ax.plot(pl_range*100000, pl-np.min(pl), 'o--')
ax.plot(pl_range*100000, pl_range*0+2.7, 'r--')
ax.set(xlabel=r'$\epsilon [10^5]$', ylabel='PL')

In [ ]:
_,ax = plt.subplots()
pl_range = np.arange(-2e-4,1e-4,1e-5)
pl = np.array([calculate_pl(pl_range_) for pl_range_ in pl_range])
ax.plot(pl_range*100000, pl-np.min(pl), 'o--', label='-2log(likelihood)')
ax.plot(pl_range*100000, pl_range*0+2.7, 'r--')
ax.set(xlabel=r'$\epsilon [10^5]$', ylabel='PL',)

pl_range_plus = np.arange(0,1e-4,1e-5)
pl_plus = np.array([calculate_pl(pl_range_) for pl_range_ in pl_range_plus])
ax.plot(pl_range_plus*100000, pl_plus-np.min(pl_plus), 'o--', label='profile likelihood')

pl_range_minus = np.arange(-2e-4,pl_range[np.argmin(pl)],1e-5)
pl_minus = np.array([calculate_pl(pl_range_) for pl_range_ in pl_range_minus])
ax.plot(pl_range_minus*100000, pl_minus-np.min(pl_minus), 'o--', label='profile likelihood')
ax.set(xlim=(-15,8), ylim=(-0.1,6))
ax.legend()
print('mean ', pl_range[np.argmin(pl)])
print('limit ', np.interp(2.71, (pl_minus-np.min(pl_minus))[::-1], pl_range_minus[::-1]), np.interp(2.71, pl_plus-np.min(pl_plus), pl_range_plus))

In [ ]:
_,ax = plt.subplots()
pl_range_plus = np.arange(0,1e-4,1e-5)
pl_plus = np.array([calculate_pl(pl_range_) for pl_range_ in pl_range_plus])
ax.plot(pl_range_plus*100000, pl_plus-np.min(pl_plus), 'o--')
ax.plot(pl_range_plus*100000, pl_range_plus*0+2.7, 'r--')

ax.set(xlabel=r'$\epsilon [10^5]$', ylabel='PL')
pl_range[np.argmin(pl)], np.interp(2.71, pl_plus-np.min(pl_plus), pl_range_plus)

In [ ]:
_,ax = plt.subplots()
pl_range_minus = np.arange(-2e-4,pl_range[np.argmin(pl)],1e-5)
pl_minus = np.array([calculate_pl(pl_range_) for pl_range_ in pl_range_minus])
ax.plot(pl_range_minus*100000, pl_minus-np.min(pl_minus), 'o--')
ax.plot(pl_range_minus*100000, pl_range_minus*0+2.7, 'r--')

ax.set(xlabel=r'$\epsilon [10^5]$', ylabel='PL')
pl_range[np.argmin(pl)], np.interp(2.71, (pl_minus-np.min(pl_minus))[::-1], pl_range_minus[::-1])